In [1]:
from pathlib import Path

from ase import units, Atoms
from ase.build import molecule
from ase.io import read, write
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect import flow
from prefect_dask import DaskTaskRunner
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

from mlip_arena.models.utils import REGISTRY, MLIPEnum
from mlip_arena.tasks.md import run as MD

## Create initial configuration

In [ ]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [ ]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [ ]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [ ]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f'{atoms.get_chemical_formula()}.extxyz', atoms)

## Run workflow

In [2]:
atoms = read("H256O128.extxyz")
print(atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


In [5]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

# cluster_kwargs = dict(
#     cores=1,
#     memory="64 GB",
#     shebang="#!/bin/bash",
#     account="matgen",
#     walltime="02:00:00",
#     job_mem="0",
#     job_script_prologue=[
#         "source ~/.bashrc",
#         "module load python",
#         "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
#     ],
#     job_directives_skip=["-n", "--cpus-per-task", "-J"],
#     job_extra_directives=[
#         "-J combustion-water",
#         "-q regular",
#         f"-N {nodes_per_alloc}",
#         "-C gpu",
#         f"-G {gpus_per_alloc}",
#         f"--exclusive",
#         # "--time-min=00:30:00",
#         # "--comment=1-00:00:00",
#         # "--signal=B:USR1@60",
#         # "--requeue",
#         # "--open-mode=append"
#     ],
#     death_timeout=86400
# )

# cluster = SLURMCluster(**cluster_kwargs)

cluster_kwargs = {
    "cores": 1,
    "memory": "64 GB",
    "shebang": "#!/bin/bash",
    "account": "matgen",
    "walltime": "00:30:00",
    "job_mem": "0",
    "job_script_prologue": [
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    "job_directives_skip": ["-n", "--cpus-per-task", "-J"],
    "job_extra_directives": [f"-N {nodes_per_alloc}", "-q debug", "-C gpu", "-J combustion-water"],
}
cluster = SLURMCluster(**cluster_kwargs)

print(cluster.job_script())
cluster.adapt(minimum_jobs=2, maximum_jobs=2)
client = Client(cluster)

#!/bin/bash

#SBATCH -A matgen
#SBATCH --mem=0
#SBATCH -t 00:30:00
#SBATCH -N 1
#SBATCH -q debug
#SBATCH -C gpu
#SBATCH -J combustion-water
source ~/.bashrc
module load python
source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.19:34913 --name dummy-name --nthreads 1 --memory-limit 59.60GiB --nanny --death-timeout 60



/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38239 instead
  warnings.warn(


In [6]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        future = MD.submit(
            atoms=atoms,
            calculator_name=model,
            calculator_kwargs=None,
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            ase_md_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            mb_velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)
    
    return [future.result() for future in futures]

In [7]:
results = combustion(atoms)

14:50:51.449 | INFO    | prefect.engine - Created flow run 'liberal-beetle' for flow 'combustion'

14:50:51.453 | INFO    | Flow run 'liberal-beetle' - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/flow-runs/flow-run/4803afc0-0472-4cf3-8d76-794fc218e2c3

14:50:51.454 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://128.55.64.19:34913

14:50:51.460 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://128.55.64.19:38239/status

14:50:51.876 | ERROR   | Flow run 'liberal-beetle' - Encountered exception during execution:
Traceback (most recent call last):
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/utilities/callables.py", line 50, in get_call_parameters
    bound_signature = inspect.signature(fn).bind(*call_args, **call_kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/inspect.py", line 3212, in bind
    return self._bind(args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/inspect.py", line 3201, in _bind
    raise TypeError(
TypeError: got an unexpected keyword argument 'mb_velocity_seed'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/engine.py", line 867, in orchestrate_flow_run
    result = await flow_call.aresult()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/_internal/concurrency/calls.py", line 327, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/_internal/concurrency/calls.py", line 352, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_997512/1920202683.py", line 6, in combustion
    future = MD.submit(
             ^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/tasks.py", line 811, in submit
    parameters = get_call_parameters(self.fn, args, kwargs)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/utilities/callables.py", line 52, in get_call_parameters
    raise ParameterBindError.from_bind_failure(fn, exc, call_args, call_kwargs)
prefect.exceptions.ParameterBindError: Error binding parameters for function 'run': got an unexpected keyword argument 'mb_velocity_seed'.
Function 'run' has signature 'atoms: 'Atoms', calculator_name: 'str | MLIPEnum', calculator_kwargs: 'dict | None', dispersion: 'str | None' = None, dispersion_kwargs: 'dict | None' = None, device: 'str | None' = None, ensemble: "Literal['nve', 'nvt', 'npt']" = 'nvt', dynamics: 'str | MolecularDynamics' = 'langevin', time_step: 'float | None' = None, total_time: 'float' = 1000, temperature: 'float | Sequence | np.ndarray | None' = 300.0, pressure: 'float | Sequence | np.ndarray | None' = None, ase_md_kwargs: 'dict | None' = None, md_velocity_seed: 'int | None' = None, zero_linear_momentum: 'bool' = True, zero_angular_momentum: 'bool' = True, traj_file: 'str | Path | None' = None, traj_interval: 'int' = 1, restart: 'bool' = True' but received args: () and kwargs: ['atoms', 'calculator_name', 'calculator_kwargs', 'ensemble', 'dynamics', 'time_step', 'ase_md_kwargs', 'total_time', 'temperature', 'pressure', 'mb_velocity_seed', 'traj_file', 'traj_interval', 'restart'].

14:50:52.106 | ERROR   | Flow run 'liberal-beetle' - Finished in state Failed('Flow run encountered an exception. ParameterBindError: Error binding parameters for function \'run\': got an unexpected keyword argument \'mb_velocity_seed\'.\nFunction \'run\' has signature \'atoms: \'Atoms\', calculator_name: \'str | MLIPEnum\', calculator_kwargs: \'dict | None\', dispersion: \'str | None\' = None, dispersion_kwargs: \'dict | None\' = None, device: \'str | None\' = None, ensemble: "Literal[\'nve\', \'nvt\', \'npt\']" = \'nvt\', dynamics: \'str | MolecularDynamics\' = \'langevin\', time_step: \'float | None\' = None, total_time: \'float\' = 1000, temperature: \'float | Sequence | np.ndarray | None\' = 300.0, pressure: \'float | Sequence | np.ndarray | None\' = None, ase_md_kwargs: \'dict | None\' = None, md_velocity_seed: \'int | None\' = None, zero_linear_momentum: \'bool\' = True, zero_angular_momentum: \'bool\' = True, traj_file: \'str | Path | None\' = None, traj_interval: \'int\' = 1, restart: \'bool\' = True\' but received args: () and kwargs: [\'atoms\', \'calculator_name\', \'calculator_kwargs\', \'ensemble\', \'dynamics\', \'time_step\', \'ase_md_kwargs\', \'total_time\', \'temperature\', \'pressure\', \'mb_velocity_seed\', \'traj_file\', \'traj_interval\', \'restart\'].')

ParameterBindError: Error binding parameters for function 'run': got an unexpected keyword argument 'mb_velocity_seed'.
Function 'run' has signature 'atoms: 'Atoms', calculator_name: 'str | MLIPEnum', calculator_kwargs: 'dict | None', dispersion: 'str | None' = None, dispersion_kwargs: 'dict | None' = None, device: 'str | None' = None, ensemble: "Literal['nve', 'nvt', 'npt']" = 'nvt', dynamics: 'str | MolecularDynamics' = 'langevin', time_step: 'float | None' = None, total_time: 'float' = 1000, temperature: 'float | Sequence | np.ndarray | None' = 300.0, pressure: 'float | Sequence | np.ndarray | None' = None, ase_md_kwargs: 'dict | None' = None, md_velocity_seed: 'int | None' = None, zero_linear_momentum: 'bool' = True, zero_angular_momentum: 'bool' = True, traj_file: 'str | Path | None' = None, traj_interval: 'int' = 1, restart: 'bool' = True' but received args: () and kwargs: ['atoms', 'calculator_name', 'calculator_kwargs', 'ensemble', 'dynamics', 'time_step', 'ase_md_kwargs', 'tot